The ipynb script is using the custom GAN network

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPU.")
    except RuntimeError as e:
        print(e)

memory_limit = 6000
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memory_limit)]
        )
        print(f"Memory limit set to {memory_limit} MB for GPU.")
    except RuntimeError as e:
        print(e)

# Load and preprocess data
dataset = np.load("/home/junze/.jupyter/Data transfer and loading/dataset_2.csv.npy")
x_mean, x_std = np.mean(dataset[:,:,0]), np.std(dataset[:,:,0])
y_mean, y_std = np.mean(dataset[:,:,1]), np.std(dataset[:,:,1])

# Normalize data
dataset_normalized = np.zeros_like(dataset)
dataset_normalized[:,:,0] = (dataset[:,:,0] - x_mean) / x_std
dataset_normalized[:,:,1] = (dataset[:,:,1] - y_mean) / y_std

# Model parameters
latent_dim = 100
seq_len = 100
output_dim = 2

generator = keras.Sequential([
    keras.layers.Input(shape=(latent_dim,)),
    keras.layers.Dense(seq_len * 8, activation='relu'),
    keras.layers.Reshape((seq_len, 8)),
    keras.layers.LSTM(16, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(output_dim))
])


discriminator = keras.Sequential([
    keras.layers.Input(shape=(seq_len, output_dim)),
    keras.layers.LSTM(16),
    keras.layers.Dense(1)
])

2024-08-30 02:51:35.780644: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-30 02:51:35.828281: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-30 02:51:35.828314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-30 02:51:35.829669: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-30 02:51:35.837781: I tensorflow/core/platform/cpu_feature_guar

Memory growth enabled for GPU.
Memory limit set to 6000 MB for GPU.


2024-08-30 02:51:39.332343: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6000 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:31:00.0, compute capability: 8.9


In [2]:

class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        noise = tf.random.normal(shape=(batch_size, self.latent_dim))

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = self.generator(noise, training=True)

            real_output = self.discriminator(real_images, training=True)
            fake_output = self.discriminator(generated_images, training=True)

            gen_loss = self.loss_fn(tf.ones_like(fake_output), fake_output)
            disc_loss = self.loss_fn(tf.ones_like(real_output), real_output) + \
                        self.loss_fn(tf.zeros_like(fake_output), fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.g_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.d_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        return {"d_loss": disc_loss, "g_loss": gen_loss}


In [3]:
gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0002),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0002),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True)
)


gan.fit(dataset_normalized, epochs=1000, batch_size=64)

noise = tf.random.normal(shape=(200, latent_dim))
generated_samples = gan.generator(noise)

generated_samples_denorm = np.zeros_like(generated_samples)
generated_samples_denorm[:,:,0] = generated_samples[:,:,0] * x_std + x_mean
generated_samples_denorm[:,:,1] = generated_samples[:,:,1] * y_std + y_mean

np.save('generated_sample_GAN1.npy', generated_samples_denorm)

Epoch 1/1000


2024-08-30 02:51:57.323264: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] function_optimizer failed: INVALID_ARGUMENT: Input 0 of node zeros_like_27 was passed float from sequential/lstm/PartitionedCall:7 incompatible with expected variant.
2024-08-30 02:51:57.537918: W tensorflow/core/common_runtime/optimize_function_graph_utils.cc:615] Ignoring multi-device function optimization failure: INVALID_ARGUMENT: Input 0 of node zeros_like_49 was passed float from sequential_1/lstm_1/PartitionedCall:7 incompatible with expected variant.
2024-08-30 02:52:00.267162: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fb2505ad1b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-30 02:52:00.267209: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-08-30 02:52:00.275303: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabli

8/8 [==============================] - 12s 474ms/step - d_loss: 1.2911 - g_loss: 0.7038
Epoch 2/1000
8/8 [==============================] - 4s 484ms/step - d_loss: 1.2700 - g_loss: 0.7062
Epoch 3/1000
8/8 [==============================] - 4s 472ms/step - d_loss: 1.2501 - g_loss: 0.7084
Epoch 4/1000
8/8 [==============================] - 4s 491ms/step - d_loss: 1.2319 - g_loss: 0.7096
Epoch 5/1000
8/8 [==============================] - 4s 483ms/step - d_loss: 1.2150 - g_loss: 0.7102
Epoch 6/1000
8/8 [==============================] - 4s 491ms/step - d_loss: 1.2000 - g_loss: 0.7095
Epoch 7/1000
8/8 [==============================] - 4s 471ms/step - d_loss: 1.1866 - g_loss: 0.7078
Epoch 8/1000
8/8 [==============================] - 4s 502ms/step - d_loss: 1.1747 - g_loss: 0.7052
Epoch 9/1000
8/8 [==============================] - 4s 519ms/step - d_loss: 1.1651 - g_loss: 0.7008
Epoch 10/1000
8/8 [==============================] - 4s 535ms/step - d_loss: 1.1576 - g_loss: 0.6953
Epoch 11/10